<a href="https://colab.research.google.com/github/judamtar67/spark_machine_learning/blob/main/ppt_hal48-55.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=0f8932fc95cfc4df0fc09616996dd02d19fcfe0488c774ab258d49186781fe74
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Movie Recommendation').getOrCreate()

In [5]:
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS

raw = spark.sparkContext.textFile("/content/drive/MyDrive/BigData/ratings.dat")

mydata = [(2, 0.01)]

mydatardd = spark.sparkContext.parallelize(mydata).map(lambda x: Rating(0, x[0], x[1]))

def parseRating(str):
    fields = str.split("::")
    assert(len(fields) == 4)
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]))

ratings = raw.map(parseRating)
totalratings = ratings.union(mydatardd)
model = ALS.train(totalratings, rank=8, iterations=5, lambda_=1.0)
products = model.recommendProducts(1, 10)

products

[Rating(user=1, product=3382, rating=4.457719776619788),
 Rating(user=1, product=989, rating=4.018027320979452),
 Rating(user=1, product=557, rating=3.9018392913456887),
 Rating(user=1, product=787, rating=3.8970494091877033),
 Rating(user=1, product=3233, rating=3.895714274317371),
 Rating(user=1, product=1830, rating=3.8954391722312773),
 Rating(user=1, product=3607, rating=3.843834348361664),
 Rating(user=1, product=3172, rating=3.82182438337599),
 Rating(user=1, product=3656, rating=3.786166561580936),
 Rating(user=1, product=2503, rating=3.785781723174142)]

In [6]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.stat import Statistics

vectorRDD = totalratings.map(lambda rating: Vectors.dense(rating.user, rating.product, rating.rating))
summary = Statistics.colStats(vectorRDD)

print(summary.mean())
print(summary.variance())
print(summary.numNonzeros())

[3024.50932404 1865.53803501    3.58156088]
[2.98741660e+06 1.20130746e+06 1.24792804e+00]
[1000209. 1000210. 1000210.]


In [7]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors

data = spark.read.text("/content/drive/MyDrive/BigData/kmeans_data.txt")
parsedData = data.rdd.map(lambda s: Vectors.dense([float(x) for x in s.value.split(' ')]))

# Convert RDD to DataFrame
parsedData = parsedData.map(lambda x: (x, )).toDF(["features"])

# Cluster the data into two classes using KMeans
numClusters = 2
numIterations = 20
kmeans = KMeans().setK(numClusters).setMaxIter(numIterations)
model = kmeans.fit(parsedData)

# Predict the cluster for each data point
predictions = model.transform(parsedData)

# Compute Within Set Sum of Squared Errors
wssse = model.summary.trainingCost
print("Within Set Sum of Squared Errors = " + str(wssse))

# Print the predicted cluster for each data point
predictions.select("prediction").show()

Within Set Sum of Squared Errors = 0.1199999999999996
+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         1|
|         1|
|         1|
+----------+



In [8]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt


# Load and parse the data
data = spark.sparkContext.textFile("/content/drive/MyDrive/BigData/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, k=2, maxIterations=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x ** 2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)

print("Within Set Sum of Squared Error = " + str(WSSSE))

# Save and load model
clusters.save(spark.sparkContext, "/content/jud_path")
sameModel = KMeansModel.load(spark.sparkContext, "/content/jud_path")

Within Set Sum of Squared Error = 0.6928203230275529
